Transformaciones

In [49]:
import numpy as np

def traslacion(c,t):
  res=[]
  res=[c[0]+t[0],c[1]+t[1],c[2]+t[2]]
  return res

def rotacion(c,theta,axis):
  P=np.matrix([[c[0]],[c[1]],[c[2]],[1]])
  theta=(theta*np.pi)/180
  if axis=="x":
    T=np.matrix([[1,0,0,0],
                 [0,np.cos(theta),-1*np.sin(theta),0],
                 [0,np.sin(theta),np.cos(theta),0],
                 [0,0,0,1]])
  elif axis=="y":
    T=np.matrix([[np.cos(theta),0,np.sin(theta),0],
                 [0,1,0,0],
                 [-1*np.sin(theta),0,np.cos(theta),0],
                 [0,0,0,1]])
  elif axis=="z":
    T=np.matrix([[np.cos(theta),-1*np.sin(theta),0,0],
                 [np.sin(theta),np.cos(theta),0,0],
                 [0,0,1,0],
                 [0,0,0,1]])
  else:
    print("Error")
    return 0
  producto=T@P
  res=[]
  for i in range(len(producto)-1):
    res.append(float(producto[i]))
  return res

def escalacion(c,esc):
  res=[]
  for i in range(len(c)):
    res.append(c[i]*esc[i])
  return res

def reflexion(c,plano):
  if plano=="xy" or plano=="yx":
    res=[c[0],c[1],c[2]*-1]
  elif plano=="yz" or plano=="zy":
    res=[-1*c[0],c[1],c[2]]
  elif plano=="xz" or plano=="zx":
    res=[c[0],-1*c[1],c[2]]
  return res

def shearing(c,a,b):
  res=[c[0]+a*c[2],c[1]+b*c[2],c[2]]
  return res

# Punto inicial
punto=[1,1,1]
print("Coordenadas originales:\n",punto)

# Traslación
puntoT=[1,1,1]
nuevoPunto=traslacion(punto,puntoT)
#print("Coordenadas de traslación:\n",puntoT,"\nNuevas coordenadas:\n",nuevoPunto)

# Rotación
angulo=90 #angulo en grados
eje="z"
nuevoPunto=rotacion(punto,angulo,eje)
print("Coordenadas de rotación de "+str(angulo)+"° sobre el eje",eje,"\n",nuevoPunto)

# Escalación
escalar=[3,3,3]
nuevoPunto=escalacion(punto,escalar)
#print("Coordenadas de escalación a: ","Sx: "+str(escalar[0])+" Sy: "+str(escalar[1])+" Sz: "+str(escalar[2]),"\n",nuevoPunto)

# Reflexión
plano="xy"
nuevoPunto=reflexion(punto,plano)
#print("Coordenadas de reflexión al plano "+str(plano)+"\n",nuevoPunto)

# Shearing
ab=[1,2]
nuevoPunto=shearing(punto,ab[0],ab[1])
#print("Coordenadas de shearing a: "+str(ab[0])+" y b: "+str(ab[1]),"\n",nuevoPunto)

Coordenadas originales:
 [1, 1, 1]
Coordenadas de rotación de 90° sobre el eje z 
 [-0.9999999999999999, 1.0, 1.0]


Raytracer

In [103]:
from PIL import Image
import numpy as np

w = 400
h = 300

def normalize(x):
    x /= np.linalg.norm(x)
    return x

def intersect_plane(O, D, P, N):
    # Return the distance from O to the intersection of the ray (O, D) with the 
    # plane (P, N), or +inf if there is no intersection.
    # O and P are 3D points, D and N (normal) are normalized vectors.
    denom = np.dot(D, N)
    if np.abs(denom) < 1e-6:
        return np.inf
    d = np.dot(P - O, N) / denom
    if d < 0:
        return np.inf
    return d

def intersect_sphere(O, D, S, R):
    # Return the distance from O to the intersection of the ray (O, D) with the 
    # sphere (S, R), or +inf if there is no intersection.
    # O and S are 3D points, D (direction) is a normalized vector, R is a scalar.
    a = np.dot(D, D)
    OS = O - S
    b = 2 * np.dot(D, OS)
    c = np.dot(OS, OS) - R * R
    disc = b * b - 4 * a * c
    if disc > 0:
        distSqrt = np.sqrt(disc)
        q = (-b - distSqrt) / 2.0 if b < 0 else (-b + distSqrt) / 2.0
        t0 = q / a
        t1 = c / q
        t0, t1 = min(t0, t1), max(t0, t1)
        if t1 >= 0:
            return t1 if t0 < 0 else t0
    return np.inf

def intersect(O, D, obj):
    if obj['type'] == 'plane':
        return intersect_plane(O, D, obj['position'], obj['normal'])
    elif obj['type'] == 'sphere':
        return intersect_sphere(O, D, obj['position'], obj['radius'])

def get_normal(obj, M):
    # Find normal.
    if obj['type'] == 'sphere':
        N = normalize(M - obj['position'])
    elif obj['type'] == 'plane':
        N = obj['normal']
    return N
    
def get_color(obj, M):
    color = obj['color']
    if not hasattr(color, '__len__'):
        color = color(M)
    return color

def trace_ray(rayO, rayD):
    # Find first point of intersection with the scene.
    t = np.inf
    for i, obj in enumerate(scene):
        t_obj = intersect(rayO, rayD, obj)
        if t_obj < t:
            t, obj_idx = t_obj, i
    # Return None if the ray does not intersect any object.
    if t == np.inf:
        return
    # Find the object.
    obj = scene[obj_idx]
    # Find the point of intersection on the object.
    M = rayO + rayD * t
    # Find properties of the object.
    N = get_normal(obj, M)
    color = get_color(obj, M)
    toL = normalize(L - M)
    toO = normalize(O - M)
    # Shadow: find if the point is shadowed or not.
    l = [intersect(M + N * .0001, toL, obj_sh) 
            for k, obj_sh in enumerate(scene) if k != obj_idx]
    if l and min(l) < np.inf:
        return
    # Start computing the color.
    col_ray = ambient
    # Lambert shading (diffuse).
    col_ray += obj.get('diffuse_c', diffuse_c) * max(np.dot(N, toL), 0) * color
    # Blinn-Phong shading (specular).
    col_ray += obj.get('specular_c', specular_c) * max(np.dot(N, normalize(toL + toO)), 0) ** specular_k * color_light
    return obj, M, N, col_ray

def add_sphere(position, radius, color):
    return dict(type='sphere', position=np.array(position), 
        radius=np.array(radius), color=np.array(color), reflection=.5)
    
def add_plane(position, normal):
    return dict(type='plane', position=np.array(position), 
        normal=np.array(normal),
        color=lambda M: (color_plane0 
            if (int(M[0] * 2) % 2) == (int(M[2] * 2) % 2) else color_plane1),
        diffuse_c=.75, specular_c=.5, reflection=.25)

# Operaciones matriciales (Bruno Pacheco)
class OperationsMat():
  def traslacion(self,c,t):
    res=[]
    res=[c[0]+t[0],c[1]+t[1],c[2]+t[2]]
    return res

  def rotacion(self,c,theta,axis):
    P=np.matrix([[c[0]],[c[1]],[c[2]],[1]])
    theta=(theta*np.pi)/180
    if axis=="x":
      T=np.matrix([[1,0,0,0],
                  [0,np.cos(theta),-1*np.sin(theta),0],
                  [0,np.sin(theta),np.cos(theta),0],
                  [0,0,0,1]])
    elif axis=="y":
      T=np.matrix([[np.cos(theta),0,np.sin(theta),0],
                  [0,1,0,0],
                  [-1*np.sin(theta),0,np.cos(theta),0],
                  [0,0,0,1]])
    elif axis=="z":
      T=np.matrix([[np.cos(theta),-1*np.sin(theta),0,0],
                  [np.sin(theta),np.cos(theta),0,0],
                  [0,0,1,0],
                  [0,0,0,1]])
      
    producto=T@P
    res=[]
    for i in range(len(producto)-1):
      res.append(float(producto[i]))
    return res

  def escalacion(self,c,esc):
    res=[]
    for i in range(len(c)):
      res.append(c[i]*esc[i])
    return res

  def reflexion(self,c,plano):
    if plano=="xy" or plano=="yx":
      res=[c[0],c[1],c[2]*-1]
    elif plano=="yz" or plano=="zy":
      res=[-1*c[0],c[1],c[2]]
    elif plano=="xz" or plano=="zx":
      res=[c[0],-1*c[1],c[2]]
    return res

  def shearing(self,c,a,b):
    res=[c[0]+a*c[2],c[1]+b*c[2],c[2]]
    return res

op=OperationsMat()
numTrans=11 #Son 9 los ejemplos de transformación y la imagen original
for i in range(numTrans): 
  print("i",i)
  # List of objects.
  color_plane0 = 1. * np.ones(3)
  color_plane1 = 0. * np.ones(3) 

  coordinates=[0,.1,1]
  if i==0:
    name="original"
    newCoordinates=coordinates
  elif i==1:
    name="traslacion"
    t=[1,1,1]
    newCoordinates=op.traslacion(coordinates,t)
  elif i==2:
    name="rotacionX"
    grados=90
    eje="x"
    newCoordinates=op.rotacion(coordinates,grados,eje)
  elif i==3:
    name="rotacionY"
    grados=90
    eje="y"
    newCoordinates=op.rotacion(coordinates,grados,eje)
  elif i==4:
    name="rotacionZ"
    grados=90
    eje="z"
    newCoordinates=op.rotacion(coordinates,grados,eje)
  elif i==5:
    name="escalacionUniforme"
    esc=[2,2,2]
    newCoordinates=op.escalacion(coordinates,esc)
  elif i==6:
    name="escalacionNoUniforme"
    esc=[2,3,4]
    newCoordinates=op.escalacion(coordinates,esc)
  elif i==7:
    name="reflexionXY"
    plano="xy"
    newCoordinates=op.reflexion(coordinates,plano)
  elif i==8:
    name="reflexionYZ"
    plano="yz"
    newCoordinates=op.reflexion(coordinates,plano)
  elif i==9:
    name="reflexionXZ"
    plano="xz"
    newCoordinates=op.reflexion(coordinates,plano)
  elif i==10:
    name="shearing"
    a=1
    b=2
    newCoordinates=op.shearing(coordinates,a,b)

  scene = [add_sphere(newCoordinates, .4, [0., 0., 1.]),
          add_plane([0., -.5, 0.], [0., 1., 0.]),
      ]

  # Light position and color.
  L = np.array([5., 5., -10.])
  color_light = np.ones(3)

  # Default light and material parameters.
  ambient = .05
  diffuse_c = 1.
  specular_c = 1.
  specular_k = 50

  depth_max = 5  # Maximum number of light reflections.
  col = np.zeros(3)  # Current color.
  O = np.array([0., 0.35, -1])  # Camera.
  Q = np.array([0., 0., 0.])  # Camera pointing to.
  img = np.zeros((h, w, 3))

  r = float(w) / h
  # Screen coordinates: x0, y0, x1, y1.
  S = (-1., -1. / r + .25, 1., 1. / r + .25)

  # Loop through all pixels.
  for i, x in enumerate(np.linspace(S[0], S[2], w)):
      if i % 10 == 0:
          print(i / float(w) * 100, "%")
      for j, y in enumerate(np.linspace(S[1], S[3], h)):
          col[:] = 0
          Q[:2] = (x, y)
          D = normalize(Q - O)
          depth = 0
          rayO, rayD = O, D
          reflection = 1.
          # Loop through initial and secondary rays.
          while depth < depth_max:
              traced = trace_ray(rayO, rayD)
              if not traced:
                  break
              obj, M, N, col_ray = traced
              # Reflection: create a new ray.
              rayO, rayD = M + N * .0001, normalize(rayD - 2 * np.dot(rayD, N) * N)
              depth += 1
              col += reflection * col_ray
              reflection *= obj.get('reflection', 1.)
          img[h - j - 1, i, :] = np.clip(col, 0, 1)

  im = Image.fromarray((255 * img).astype(np.uint8), "RGB")
  im.save(name+".png")

i 0
0.0 %
2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
17.5 %
20.0 %
22.5 %
25.0 %
27.500000000000004 %
30.0 %
32.5 %
35.0 %
37.5 %
40.0 %
42.5 %
45.0 %
47.5 %
50.0 %
52.5 %
55.00000000000001 %
57.49999999999999 %
60.0 %
62.5 %
65.0 %
67.5 %
70.0 %
72.5 %
75.0 %
77.5 %
80.0 %
82.5 %
85.0 %
87.5 %
90.0 %
92.5 %
95.0 %
97.5 %
i 1
0.0 %
2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
17.5 %
20.0 %
22.5 %
25.0 %
27.500000000000004 %
30.0 %
32.5 %
35.0 %
37.5 %
40.0 %
42.5 %
45.0 %
47.5 %
50.0 %
52.5 %
55.00000000000001 %
57.49999999999999 %
60.0 %
62.5 %
65.0 %
67.5 %
70.0 %
72.5 %
75.0 %
77.5 %
80.0 %
82.5 %
85.0 %
87.5 %
90.0 %
92.5 %
95.0 %
97.5 %
i 2
0.0 %
2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
17.5 %
20.0 %
22.5 %
25.0 %
27.500000000000004 %
30.0 %
32.5 %
35.0 %
37.5 %
40.0 %
42.5 %
45.0 %
47.5 %
50.0 %
52.5 %
55.00000000000001 %
57.49999999999999 %
60.0 %
62.5 %
65.0 %
67.5 %
70.0 %
72.5 %
75.0 %
77.5 %
80.0 %
82.5 %
85.0 %
87.5 %
90.0 %
92.5 %
95.0 %
97.5 %
i 3
0.0 %
2.5 %
5.0 %
7.5 %
10.0 %
12.5 